In [1]:
234

234

In [12]:

from dotenv import load_dotenv
import os
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor

# Load environment variables
load_dotenv()

from langchain_cohere import ChatCohere



from dotenv import load_dotenv
import os
from langgraph.prebuilt import create_react_agent

In [13]:
chat_model = ChatCohere(
    cohere_api_key=os.getenv("COHERE_API_KEY"),
    model="command-r-plus",  # Cohere's latest model
    temperature=0.7
)


In [14]:
from langchain_core.tools import tool

@tool
def book_hotel_tool(hotel_name: str) -> str:
    """Book a hotel reservation
    
    Args:
        hotel_name: Name of the hotel to book
    """
    return f"Successfully booked a stay at {hotel_name}."

@tool  
def book_flight_tool(from_airport: str, to_airport: str) -> str:
    """Book a flight ticket
    
    Args:
        from_airport: Departure airport code
        to_airport: Arrival airport code
    """
    return f"Successfully booked a flight from {from_airport} to {to_airport}."


In [19]:
# First, let's fix the agent creation
flight_assistant = create_react_agent(
    model=chat_model,
    tools=[book_flight_tool],
    prompt="You are a flight booking assistant",
    name="flight_assistant"  # Added name parameter
)

hotel_assistant = create_react_agent(
    model=chat_model,
    tools=[book_hotel_tool],
    prompt="You are a hotel booking assistant",
    name="hotel_assistant"  # Added name parameter
)


#### LLM based Routing example

In [27]:
from langchain_core.prompts import ChatPromptTemplate
from typing import Annotated, TypedDict, Literal
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langgraph.graph import StateGraph, START, END

# Define the state - this is passed between all nodes
class TravelState(TypedDict):
    messages: Annotated[list[BaseMessage], "The conversation messages"]
    current_agent: Annotated[str, "Currently active agent"]

# Define a routing prompt template
router_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a smart routing assistant that directs user requests to the appropriate service:
    - flight_assistant: for anything related to flight bookings, air travel, or airports
    - hotel_assistant: for anything related to hotel bookings, accommodations, or stays
    - END: if the request doesn't match either service
    
    Respond ONLY with one of these three options: flight_assistant, hotel_assistant, or END"""),
    ("user", "{message}")
])

def llm_route_request(state: TravelState) -> Literal["flight_assistant", "hotel_assistant", END]:
    """Smart router using LLM to understand and route requests"""
    print(f"🔄 ROUTER: Current state has {len(state['messages'])} messages")
    
    if not state["messages"]:
        print("🔄 ROUTER: No messages, ending")
        return END
    
    last_message = state["messages"][-1]
    print(f"🔄 ROUTER: Last message type: {type(last_message).__name__}")
    print(f"🔄 ROUTER: Message content: {last_message.content}")
    
    # Use LLM to determine routing
    chain = router_prompt | chat_model
    response = chain.invoke({"message": last_message.content})
    
    print("----------------------------------------------------------------------")
    print(f"response --> {response}")
    print("_________________________________________________________________________")
    route = response.content.strip().lower()
    
    print(f"🤖 LLM Router decided: {route}")
    
    if route == "flight_assistant":
        print("✈️  ROUTER: Routing to flight_assistant")
        return "flight_assistant"
    elif route == "hotel_assistant":
        print("🏨 ROUTER: Routing to hotel_assistant")
        return "hotel_assistant"
    else:
        print("❓ ROUTER: No clear match, ending")
        return END

# Update the workflow to use the LLM router
# ... existing workflow setup code ...
# Wrapper functions for your agents (required for StateGraph)
def flight_node(state: TravelState):
    """Flight assistant node"""
    print("✈️  FLIGHT_NODE: Starting flight assistant")
    print(f"✈️  FLIGHT_NODE: Input state messages: {len(state['messages'])}")
    
    # Call your existing flight assistant
    result = flight_assistant.invoke({"messages": state["messages"]})
    
    print(f"✈️  FLIGHT_NODE: Flight assistant returned {len(result['messages'])} messages")
    
    # Return updated state
    return {
        "messages": result["messages"],
        "current_agent": "flight_assistant"
    }

def hotel_node(state: TravelState):
    """Hotel assistant node"""
    print("🏨 HOTEL_NODE: Starting hotel assistant")
    print(f"🏨 HOTEL_NODE: Input state messages: {len(state['messages'])}")
    
    # Call your existing hotel assistant  
    result = hotel_assistant.invoke({"messages": state["messages"]})
    
    print(f"🏨 HOTEL_NODE: Hotel assistant returned {len(result['messages'])} messages")
    
    # Return updated state
    return {
        "messages": result["messages"],
        "current_agent": "hotel_assistant"
    }


# Create the StateGraph
print("🏗️  Creating StateGraph...")
workflow = StateGraph(TravelState)

# Add nodes
print("🏗️  Adding nodes...")
workflow.add_node("flight_assistant", flight_node)
workflow.add_node("hotel_assistant", hotel_node)

workflow.add_conditional_edges(
    START,
    llm_route_request,  # Use the LLM router instead of keyword matching
    {
        "flight_assistant": "flight_assistant",
        "hotel_assistant": "hotel_assistant",
        END: END
    }
)



# After each agent runs, go to END
workflow.add_edge("flight_assistant", END)
workflow.add_edge("hotel_assistant", END)

# Compile the graph
print("🏗️  Compiling graph...")
app = workflow.compile()

# Test it!
print("\n🧪 TESTING THE GRAPH:")

# Test 1: Flight request
print("\n--- Test 1: Flight Request ---")
initial_state = {
    "messages": [HumanMessage(content="I need to book a flight from LAX to JFK")],
    "current_agent": ""
}
print(f"🧪 Initial state: {initial_state}")

result = app.invoke(initial_state)
print(f"🧪 Final result: {result}")

# Test 2: Hotel request  
print("\n--- Test 2: Hotel Request ---")
initial_state = {
    "messages": [HumanMessage(content="I need to book a hotel in New York")],
    "current_agent": ""
}
print(f"🧪 Initial state: {initial_state}")

result = app.invoke(initial_state)
print(f"🧪 Final result: {result}")


# ... rest of the existing code ...

🏗️  Creating StateGraph...
🏗️  Adding nodes...
🏗️  Compiling graph...

🧪 TESTING THE GRAPH:

--- Test 1: Flight Request ---
🧪 Initial state: {'messages': [HumanMessage(content='I need to book a flight from LAX to JFK', additional_kwargs={}, response_metadata={})], 'current_agent': ''}
🔄 ROUTER: Current state has 1 messages
🔄 ROUTER: Last message type: HumanMessage
🔄 ROUTER: Message content: I need to book a flight from LAX to JFK
----------------------------------------------------------------------
response --> content='flight_assistant' additional_kwargs={'id': '142244ff-abf5-454c-9bea-28ddaa26db88', 'finish_reason': 'COMPLETE', 'content': 'flight_assistant', 'token_count': {'input_tokens': 102.0, 'output_tokens': 3.0}} response_metadata={'id': '142244ff-abf5-454c-9bea-28ddaa26db88', 'finish_reason': 'COMPLETE', 'content': 'flight_assistant', 'token_count': {'input_tokens': 102.0, 'output_tokens': 3.0}} id='run--ab65deab-d982-46c5-b115-74180e711024-0' usage_metadata={'input_tokens': 

#### Simple routing function - decides which agent to call

In [25]:
# from typing import Annotated, TypedDict, Literal
# from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
# from langgraph.graph import StateGraph, START, END

# # Define the state - this is passed between all nodes
# class TravelState(TypedDict):
#     messages: Annotated[list[BaseMessage], "The conversation messages"]
#     current_agent: Annotated[str, "Currently active agent"]

# # Simple routing function - decides which agent to call
# def route_request(state: TravelState) -> Literal["flight_assistant", "hotel_assistant", END]:
#     """Simple router based on keywords in the last message"""
#     print(f"🔄 ROUTER: Current state has {len(state['messages'])} messages")
    
#     if not state["messages"]:
#         print("🔄 ROUTER: No messages, ending")
#         return END
    
#     last_message = state["messages"][-1]
#     print(f"🔄 ROUTER: Last message type: {type(last_message).__name__}")
#     print(f"🔄 ROUTER: Message content: {last_message.content}")
    
#     # Simple keyword-based routing
#     content = last_message.content.lower()
    
#     if "flight" in content or "airport" in content or "fly" in content:
#         print("✈️  ROUTER: Routing to flight_assistant")
#         return "flight_assistant"
#     elif "hotel" in content or "room" in content or "stay" in content:
#         print("🏨 ROUTER: Routing to hotel_assistant") 
#         return "hotel_assistant"
#     else:
#         print("❓ ROUTER: No clear match, ending")
#         return END

# # Wrapper functions for your agents (required for StateGraph)
# def flight_node(state: TravelState):
#     """Flight assistant node"""
#     print("✈️  FLIGHT_NODE: Starting flight assistant")
#     print(f"✈️  FLIGHT_NODE: Input state messages: {len(state['messages'])}")
    
#     # Call your existing flight assistant
#     result = flight_assistant.invoke({"messages": state["messages"]})
    
#     print(f"✈️  FLIGHT_NODE: Flight assistant returned {len(result['messages'])} messages")
    
#     # Return updated state
#     return {
#         "messages": result["messages"],
#         "current_agent": "flight_assistant"
#     }

# def hotel_node(state: TravelState):
#     """Hotel assistant node"""
#     print("🏨 HOTEL_NODE: Starting hotel assistant")
#     print(f"🏨 HOTEL_NODE: Input state messages: {len(state['messages'])}")
    
#     # Call your existing hotel assistant  
#     result = hotel_assistant.invoke({"messages": state["messages"]})
    
#     print(f"🏨 HOTEL_NODE: Hotel assistant returned {len(result['messages'])} messages")
    
#     # Return updated state
#     return {
#         "messages": result["messages"],
#         "current_agent": "hotel_assistant"
#     }

# # Create the StateGraph
# print("🏗️  Creating StateGraph...")
# workflow = StateGraph(TravelState)

# # Add nodes
# print("🏗️  Adding nodes...")
# workflow.add_node("flight_assistant", flight_node)
# workflow.add_node("hotel_assistant", hotel_node)

# # Add edges
# print("🏗️  Adding edges...")
# # Start by routing the initial request
# workflow.add_conditional_edges(
#     START,  # From the start
#     route_request,  # Use this function to decide where to go
#     {
#         "flight_assistant": "flight_assistant",  # If returns "flight_assistant", go to flight node
#         "hotel_assistant": "hotel_assistant",    # If returns "hotel_assistant", go to hotel node  
#         END: END  # If returns END, finish
#     }
# )

# # After each agent runs, go to END
# workflow.add_edge("flight_assistant", END)
# workflow.add_edge("hotel_assistant", END)

# # Compile the graph
# print("🏗️  Compiling graph...")
# app = workflow.compile()

# # Test it!
# print("\n🧪 TESTING THE GRAPH:")

# # Test 1: Flight request
# print("\n--- Test 1: Flight Request ---")
# initial_state = {
#     "messages": [HumanMessage(content="I need to book a flight from LAX to JFK")],
#     "current_agent": ""
# }
# print(f"🧪 Initial state: {initial_state}")

# result = app.invoke(initial_state)
# print(f"🧪 Final result: {result}")

# # Test 2: Hotel request  
# print("\n--- Test 2: Hotel Request ---")
# initial_state = {
#     "messages": [HumanMessage(content="I need to book a hotel in New York")],
#     "current_agent": ""
# }
# print(f"🧪 Initial state: {initial_state}")

# result = app.invoke(initial_state)
# print(f"🧪 Final result: {result}")

🏗️  Creating StateGraph...
🏗️  Adding nodes...
🏗️  Adding edges...
🏗️  Compiling graph...

🧪 TESTING THE GRAPH:

--- Test 1: Flight Request ---
🧪 Initial state: {'messages': [HumanMessage(content='I need to book a flight from LAX to JFK', additional_kwargs={}, response_metadata={})], 'current_agent': ''}
🔄 ROUTER: Current state has 1 messages
🔄 ROUTER: Last message type: HumanMessage
🔄 ROUTER: Message content: I need to book a flight from LAX to JFK
✈️  ROUTER: Routing to flight_assistant
✈️  FLIGHT_NODE: Starting flight assistant
✈️  FLIGHT_NODE: Input state messages: 1
✈️  FLIGHT_NODE: Flight assistant returned 4 messages
🧪 Final result: {'messages': [HumanMessage(content='I need to book a flight from LAX to JFK', additional_kwargs={}, response_metadata={}, id='3f84bd14-52d3-40a4-b2a5-f82b3eff8611'), AIMessage(content='I will book a flight from LAX to JFK.', additional_kwargs={'id': '78da1a8a-0148-4ebf-932c-fae0991ccf18', 'finish_reason': 'TOOL_CALL', 'tool_plan': 'I will book a flig